# [Fe/H] Metallicity Profiles (Gas)

In [95]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as ss

In [96]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.parser import parse

In [97]:
figure_setup()
settings = Settings()

In [98]:
RXY_RANGE: tuple = (0, 30)
N_BINS: int = 30
SNAPSHOTS = [52, 66, 79, 92, 104, 116, 127]

In [100]:
colors = plt.cm.winter(np.linspace(0, 1, len(SNAPSHOTS)))

In [99]:
def read_data(simulation: str, of: str, to: str) -> tuple:
    s = Snapshot(simulation=simulation, loadonlytype=[0])
    s.add_extra_coordinates()
    s.add_metal_abundance(of=of, to=to)

    is_gas = (s.type == 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df[f"[{of}/{to}]"] = s.metal_abundance[f"{of}/{to}"][
        is_gas & is_main_obj]
    df["CylindricalRadius_ckpc"] = s.rho[is_gas & is_main_obj]

    # Remove invalid data
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    return s.time, df

In [104]:
fig = plt.figure(figsize=(7, 8))
gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flatten():
    ax.tick_params(which='both', direction="in")
    if ax == axs[-1, -1]: ax.axis("off")
    ax.set_xlim(RXY_RANGE)
    ax.set_xticks([5, 10, 15, 20, 25])
    ax.set_ylim(-0.8, 0.8)
    ax.set_yticks([-0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6])
    ax.set_axisbelow(True)
    if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
        ax.set_xlabel(r"$r_{xy}$ [ckpc]")
        ax.tick_params(labelbottom=True)
    if ax.get_subplotspec().is_first_col():
        ax.set_ylabel("[Fe/H]")

for i in range(len(settings.groups["Included"])):
    ax = axs.flatten()[i]
    galaxy = settings.groups["Included"][i]
    label = f"Au{galaxy}"
    for j, snapnum in enumerate(SNAPSHOTS):
        simulation = f"au{galaxy}_or_l4_s{snapnum}"
        time, df = read_data(simulation=simulation, of="Fe", to='H')

        stat, bin_edges, _ = ss.binned_statistic(
            x=df["CylindricalRadius_ckpc"],
            values=df["[Fe/H]"],
            statistic=np.nanmean,
            bins=N_BINS,
            range=RXY_RANGE,
            )
        bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
        ax.plot(bin_centers, stat, label=np.round(time, 1), color=colors[j])

    if ax == axs[-1, -2]:
        ax.legend(loc="center", bbox_to_anchor=(1.5, 0.3),
                  title="Time [Gyr]", ncol=2, fontsize=4.0, framealpha=0)

    ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$",
            size=6.0, transform=ax.transAxes, ha='left', va='top')

    fig.savefig("../images/FeH_metallicity_profiles/included_gas.pdf")

plt.close(fig)